In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from UV_views import *
from Y_views import *
from efficiency import *
%matplotlib inline

In [2]:
all_hits = pd.read_csv('Data/strawtubesPoint_decoded_hits.csv', delimiter=',')
all_hits['Index'] = all_hits.index
all_hits = all_hits[['Index', 'X', 'Y', 'Z', 'TrackID', 'Time', 'StatNb', 'ViewNb', 'PlaneNb', 'LayerNb', 'StrawNb', 'dist2Wire', 'event', 'k']]
all_hits.head()

,Index,X,Y,Z,TrackID,Time,StatNb,ViewNb,PlaneNb,LayerNb,StrawNb,dist2Wire,event,k
0,0,8.883409,-61.202305,-1974.859985,3,232.044861,5,0,0,0,319,0.274822,0,0
1,1,8.858191,-61.225544,-1973.742188,3,232.074081,5,0,0,1,320,0.582455,0,1
2,2,8.824563,-61.256523,-1972.251953,3,232.118286,5,0,1,0,319,0.111252,0,2
3,3,8.632817,-61.432972,-1963.756592,3,232.406479,5,1,0,1,320,0.164412,0,3
4,4,8.573978,-61.487133,-1961.149292,3,232.488541,5,1,1,1,320,0.226827,0,4


In [3]:
def combine(tracks_yz, tracks_xz, ind):
    pairs = {}
    s = tracks_xz.keys()
    for i in s:
        for j in s:
            if (i != j) and (i not in pairs) and (j not in pairs):
                z_x = (tracks_xz[i][1] - tracks_xz[j][1]) / (tracks_xz[j][0] - tracks_xz[i][0])
                if (z_x > -2000 and z_x < 2598):
                    if (i / 10000 == j / 10000):
                        pairs[i] = j
                        pairs[j] = i
                    else:
                        z_y = (tracks_yz[i / 10000][1] - tracks_yz[j / 10000][1]) / (tracks_yz[j / 10000][0] - tracks_yz[i / 10000][0])
                        if (z_y > z_x + 100 and z_y < z_x - 100):
                            pairs[i] = j
                            pairs[j] = i
    return pairs

In [4]:
def matching(tracks_yz_before, tracks_xz_before, tracks_yz_after, tracks_xz_after):
    match = {}
    for i in tracks_xz_before:
        for j in tracks_xz_after:
            if (i not in match.keys() and j not in match.values()):
                x_before = tracks_xz_before[i][0] * 3000 + tracks_xz_before[i][1]
                x_after = tracks_xz_after[j][0] * 3000 + tracks_xz_after[j][1]
                if (x_before > x_after - 20 and x_before < x_after + 20):
                    y_before = tracks_yz_before[i / 10000][0] * 3000 + tracks_yz_before[i / 10000][1]
                    y_after = tracks_yz_after[j / 10000][0] * 3000 + tracks_yz_after[j / 10000][1]
                    if (y_before < y_after + 2 and y_before > y_after - 2):
                        match[i] = j
    return match

In [6]:
for i in range(2000, 3000):
    event = all_hits[all_hits['event'] == i]
    tracks_yz_1_2, linking_table_yz_1_2 = loop_yz(event, 7, 0.85, 0)
    if len(tracks_yz_1_2) < 2:
        continue
    tracks_xz_1_2, linking_table_xz_1_2, Xs = loop_xz(event, tracks_yz_1_2, linking_table_yz_1_2, 6, 15, 0)
    if len(tracks_xz_1_2) < 2:
        continue
    tracks_yz_3_4, linking_table_yz_3_4 = loop_yz(event, 7, 0.85, 1)
    if len(tracks_yz_3_4) < 2:
        continue
    tracks_xz_3_4, linking_table_xz_3_4, Xs = loop_xz(event, tracks_yz_3_4, linking_table_yz_3_4, 6, 15, 1)
    if len(tracks_xz_3_4) < 2:
        continue
    pairs_1_2 = combine(tracks_yz_1_2, tracks_xz_1_2, 0)
    if len(pairs_1_2) < 1:
        continue
    pairs_3_4 = combine(tracks_yz_3_4, tracks_xz_3_4, 0)
    if len(pairs_3_4) < 1:
        continue
    match = matching(tracks_yz_1_2, tracks_xz_1_2, tracks_yz_3_4, tracks_xz_3_4)
    if len(match) > 0:
        print "index:" + str(i)